In [6]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - There is no [win32] chromedriver for browser 85.0.4183 in cache
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/85.0.4183.87/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\conex_3\.wdm\drivers\chromedriver\win32\85.0.4183.87]
